<a href="https://colab.research.google.com/github/AdamsD02/Music-Genre-Classification/blob/main/mgc_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone "https://github.com/AdamsD02/Music-Genre-Classification.git"


Cloning into 'Music-Genre-Classification'...
remote: Enumerating objects: 1018, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1018 (delta 2), reused 0 (delta 0), pack-reused 1012 (from 2)
Receiving objects: 100% (1018/1018), 1.13 GiB | 30.30 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (1003/1003), done.


In [4]:
%cd "Music-Genre-Classification"

/content/Music-Genre-Classification


In [98]:
%ls

features_30_sec.csv  genres/  mgc_code.ipynb  README.md


### Install Librosa library for audio analysis and manipulation

In [ ]:
pip install librosa

In [99]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


### Import Libraries

In [100]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


## Read .csv file containing data for Classifying Genres

In [101]:
df = pd.read_csv("/content/Music-Genre-Classification/features_30_sec.csv")
# df.head()
# list dataset features
print(df.columns.tolist())

['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var', 'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var', 'label']


### Manipulate dataframe to:
*   Separate target(X) and features(y)
*   Split into traning and testing (80-20)

In [103]:
X = df.iloc[:, 2:59] # chroma_stft_mean to mfcc20_var

# Encode non-numerical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['label'])

print(X.shape)
print(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print("X_train size: ", len(X_train))
print("X_test size: ", len(X_test))

(1000, 57)
[0 1 2 3 4 5 6 7 8 9]
X_train size:  800
X_test size:  200


### Use LightGBM as Model Classification and training

In [117]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
# clf = lgb.LGBMClassifier(force_col_wise=False)
clf.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000914 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14312
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 57
[LightGBM] [Info] Start training from score -2.290163
[LightGBM] [Info] Start training from score -2.315164
[LightGBM] [Info] Start training from score -2.340806
[LightGBM] [Info] Start training from score -2.290163
[LightGBM] [Info] Start training from score -2.277892
[LightGBM] [Info] Start training from score -2.327903
[LightGBM] [Info] Start training from score -2.277892
[LightGBM] [Info] Start training from score -2.353878
[LightGBM] [Info] Start training from score -2.253795
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

LGBMClassifier()

### Testing and Predictions

In [118]:
# predict the results
y_pred = clf.predict(X_test)

predicted_genres = le.inverse_transform(y_pred)

mse = np.mean((y_pred - y_test) ** 2)
count = 0
print("Mse: ", mse)
# print(y_pred)
for i in range(len(y_pred)):
  if y_pred[i] - y_test[i] == 0:
    count += 1
  # print("Predicted: ", predicted_genres[i], "\t| Actual: ", le.inverse_transform([y_test[i]]))

print("Accuracy: ", count/len(y_pred))


Mse:  7.76
Accuracy:  0.73
